In [1]:
!pip install cvlib

     ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/13.1 MB 7.5 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/13.1 MB 8.4 MB/s eta 0:00:02
     --------------- ------------------------ 5.0/13.1 MB 8.4 MB/s eta 0:00:01
     -------------------- ------------------- 6.8/13.1 MB 8.6 MB/s eta 0:00:01
     -------------------------- ------------- 8.7/13.1 MB 8.5 MB/s eta 0:00:01
     --------------------------------- ------ 11.0/13.1 MB 8.8 MB/s eta 0:00:01
     -------------------------------------- - 12.6/13.1 MB 8.9 MB/s eta 0:00:01
     ---------------------------------------- 13.1/13.1 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with s

In [5]:
!pip install tensorflow==2.16.2

  Using cached tensorflow-2.16.2-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
Using cached tensorflow-2.16.2-cp312-cp312-win_amd64.whl (2.1 kB)


In [60]:
import cvlib as cv # 얼굴 탐지 도구
import cv2
import matplotlib.pyplot as plt

### 이미지에서 얼굴 탐지

In [18]:
img = cv2.imread('./data/pic.png')
faces, conf = cv.detect_face(img)
print(faces) # 얼굴이 있는 좌상단, 우하단 좌표 표기
print(conf) # 얼굴의 신뢰도

cv2.imshow('pic', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[22, 191, 123, 317], [238, 103, 329, 207], [464, 65, 557, 170]]
[0.9990305, 0.9981894, 0.97535056]


In [20]:
# 이미지에 사각형 도형 출력하기
import numpy as np
img_bg = np.zeros((480, 640, 3))
line_color = [0, 255, 0] # 사각형 색깔
line_thick = 4 # 사각형 선의 두깨

#cv2.rectangle(이미지, 좌상단, 우하단, 색상, 두깨)
cv2.rectangle(img_bg, (0,240), (240,480), line_color, line_thick)
cv2.imshow('img_bg1', img_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# 첫번째 사람의 얼굴 좌표
faces[0]
# 첫번째 얼굴의 얼굴 정확도
con

[22, 191, 123, 317]

In [58]:
img = cv2.imread('./data/pic.png')
faces, conf = cv.detect_face(img)
color = [0, 255, 255]
line_th = 3

# 여러명의 얼굴에 사각형
'''
for i in range(len(faces)):
    # i 0, 1, 2
    # putText(이미지, 텍스트, 위치, 글꼴, 글씨크기,
    cv2.putText(img, f'{conf[i]}', (faces[i][0],faces[i][1]-10), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    cv2.rectangle(img, (faces[i][0],faces[i][1]), (faces[i][2],faces[i][3]), color, line_th)
'''

# 감지된 얼굴에 사각형을 씌워주고, 정확도를 출력하는 코드
for (x,y,x2,y2), c in zip(faces, conf):
    cv2.putText(img, f'{c:.2f}', (x,y-10), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    cv2.rectangle(img, (x,y), (x2,y2), color, line_th)

cv2.imshow('pic', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
(x,y,x2,y2), c = list(zip(faces, conf))[0]
x
y
c

0.9990305

In [64]:
cap = cv2.VideoCapture(0)
color = [0, 255, 255]
line_th = 3
cnt = 0

while True:
    try:
        ch, frame = cap.read() # ch : 성공여부, frame : 이미지

        # cv2.flip(대상이미지, 1) : 좌우 반전
        flip_frame = cv2.flip(frame, 1)
        
        # 실패할 경우 while문 멈추기
        if not ch: # ch가 false일 경우
            print('동영상 재생 끝')
            break

        # 얼굴 탐지
        faces, conf = cv.detect_face(flip_frame)
        # faces : 각 얼굴의 좌표
        # conf : 얼굴의 정확도

        # cv2.putText(이미지, 텍스트, 위치, 글꼴)
        for (x,y,x2,y2), c in zip(faces, conf):
            cv2.putText(flip_frame, f'{c:.2f}', (x,y-10), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
            cv2.rectangle(flip_frame, (x,y), (x2,y2), color, line_th)
        cv2.imshow('video', flip_frame)
        key = cv2.waitKey(10)
        if key == ord('q'):
            print('사용자에 의해서 영상이 종료')
            break

        # 사진찍기. 프레임 저장
        if key == ord('w'):
            cnt += 1
            print(f'./data/screenshot{cnt}.png 사진저장')
            cv2.imwrite(f'./data/screenshot{cnt}.png', flip_frame, params = [cv2.IMWRITE_PNG_COMPRESSION, 0])

    except:
        print('카메라 읽기 싪패 or 영상 끝')
        cap.release() # 카메라 연결 해제
        cv2.destroyAllWindows()
        # try문 안에 있는 while문에서 break로 나갔을 경우
cap.release()
cv2.destroyAllWindows()

./data/screenshot1.png 사진저장
./data/screenshot2.png 사진저장
./data/screenshot3.png 사진저장
사용자에 의해서 영상이 종료


In [62]:
# 얼굴 인식이 안됐을때, 에러가 발생하는 경우가 존재.
# 카메라가 읽도록 재실행 연결
# 인식이 안될 경우 카메라가 꺼지도록 설정
# try-except : 예외처리
try:
    print(wefasdf)
except:
    print('선언되지 않은 변수')

선언되지 않은 변수
